In [3]:
import numpy as np
import cv2
import sklearn.model_selection as sk
from analysis import parse_protobufs
from ipdb import set_trace as debug
import os
import tensorflow as tf

In [4]:
class struct():
	pass

In [5]:
def parse_data(save=True):

	data_dict_x = {}#rgb classified by object
	data_dict_d = {}#rgb classified by object    
	data_dict_y = {}#rgb classified by object
	batch_dict = {}
	save_path_train = '/home/zhouzixuan/notebooks/proj_new/data/train/'
	save_path_test =  '/home/zhouzixuan/notebooks/proj_new/data/test/'
	# example data extraction of x value of object/item 0 in training example 0: data.states[0].items[0].x
	num_examples = 18000 # number or screenshots
	num_items = []  # number of items in each example
	labels = []
	labels_rgb = []
	X_rgb = np.empty([0,299,299,3])
    
	X_d = np.empty([0,299,299])
	batch_size = 32
	path = struct()
	path.data_name = '_SessionStateData.proto'
	path.data_folder = 'TeleOpVRSession_2018-02-05_15-44-11/'
	data0 = parse_protobufs(path)
	path.data_folder = 'TeleOpVRSession_2018-03-07_14-38-06_Camera1/'
	data1 = parse_protobufs(path)    
	path.data_folder = 'TeleOpVRSession_2018-03-07_14-38-06_Camera2/'
	data2 = parse_protobufs(path)    
	path.data_folder = 'TeleOpVRSession_2018-03-07_14-38-06_Camera3/'
	data3 = parse_protobufs(path)    
	path.data_folder = 'TeleOpVRSession_2018-03-07_14-38-06_Camera4/'
	data4 = parse_protobufs(path)    
	# format labels into n x 6 array
	for i in range(15844,18000+1):
		print i
		path = struct()
		if i < 10000:
			t = i
			data = data0
		else:
			if i < 12000:
				t = i - 10000
				data = data1
			else:
				if i < 14000:
					t = i - 12000                
					data = data2
				else:
					if i < 16000:
						t = i - 14000                                 
						data = data3
					else:
						t = i - 16000                         
						data = data4
		num_items.append(len(data.states[t].items))
		img_name = str(data.states[t].snapshot.name)
		depth_name = img_name[:-4] + '-Depth.jpg'

		# read in rgb and depth images and add a new axis to them to indicate which snapshot index for each image
		rgb_img = np.expand_dims(cv2.imread(img_name, 1), axis=0)
		depth_img = np.expand_dims(cv2.imread(depth_name, 0), axis = 0)
        ###########
		for j in range(num_items[i - 15844]):
            
			item_id = str(data.states[t].items[j].id)
			item_id_int = data.states[t].items[j].id            
			if item_id_int != 35: 
				continue
			else:
				print 666
			'''
			RGB label, classified by name
			input label (X)
			D label, classified by name
			input label (X)
			'''            
			if item_id not in data_dict_x:
				data_dict_x[item_id] = np.empty([0,299,299,3])
				data_dict_d[item_id] = np.empty([0,299,299])                
			data_dict_x[item_id] = np.vstack([data_dict_x[item_id], rgb_img])
			data_dict_d[item_id] = np.vstack([data_dict_d[item_id], depth_img])
            
			'''
			RGB-D label, classified by name
			Batch split
			'''
			if item_id not in batch_dict:
				batch_dict[item_id] = 62
                              
			# Output label (Y)
			rlabel = data.states[t].items[j] 
			current_label = [data.states[t].snapshot.name, rlabel.x, rlabel.y, rlabel.z, rlabel.roll, rlabel.pitch, rlabel.yaw]
			#print data.states[i].items[j].id
			labels.append(current_label)
			'''
			RGB label
			'''
			current_label_rgb = [rlabel.x, rlabel.y, rlabel.z]
			labels_rgb.append(current_label_rgb)
            
			'''
			RGB label, classified by name
			Output label (Y)
			'''
            
			if item_id not in data_dict_y:
				data_dict_y[item_id] = []
			data_dict_y[item_id].append(current_label_rgb)
            
			if len(data_dict_x[item_id]) == batch_size:
				batch = batch_dict[item_id]
				if i % 10 != 0:
					tmp_path = save_path_train
				else:
					tmp_path = save_path_test                   
				if not os.path.exists(tmp_path):
					os.makedirs(tmp_path)
                           
				np.save(tmp_path +"/"+ str(batch) +"_x.npy", data_dict_x[item_id])
				np.save(tmp_path +"/"+ str(batch) +"_d.npy", data_dict_d[item_id])       
				np.save(tmp_path +"/"+ str(batch) +"_y.npy", np.array(data_dict_y[item_id]))  
                
				train_path = "/home/zhouzixuan/notebooks/proj_new/3ddata/train/"
				test_path = "/home/zhouzixuan/notebooks/proj_new/3ddata/test/"
				if not os.path.exists(train_path):
					os.makedirs(train_path)
				if not os.path.exists(test_path):
					os.makedirs(test_path)
                    
				d_batch = data_dict_d[item_id]
				x_batch = data_dict_x[item_id]
				d_round = np.floor(d_batch/25.5)
				sess = tf.InteractiveSession()
				v = tf.transpose(tf.one_hot(d_round, depth=10, axis=2, on_value=1.0, off_value=0.0), perm=[0,1,3,2])
				v = v.eval()
				combine = np.empty([32, 299, 299, 3, 0])
				for i in range(10):
					i = 1
					v_tmp = v[:,:,:,i]
					v_tmp = np.transpose(np.broadcast_to(v_tmp,(3, 32, 299, 299)),(1,2,3,0))
					v_tmp = v_tmp == 1
					x_tmp = np.multiply(x_batch, v_tmp)
					x_cur = np.expand_dims(x_tmp, axis = 4)
					combine = np.concatenate((combine, x_cur), axis=4)
				np.save(train_path +"/"+ str(batch) +".npy", combine)       
				np.save(test_path +"/"+ str(batch) +"_y.npy", np.array(data_dict_y[item_id]))  
                
				data_dict_x[item_id] = np.empty([0,299,299,3])
				data_dict_d[item_id] = np.empty([0,299,299])                
				data_dict_y[item_id] = []               
				batch_dict[item_id] = 1 + batch
                       


In [6]:
if __name__ == '__main__':
	parse_data(save=True)

15844
15845
15846
15847
15848
15849
15850
15851
15852
15853
15854
15855
15856
15857
15858
15859
15860
666
15861
15862
15863
15864
15865
666
15866
15867
666
15868
666
15869
15870
15871
666
15872
15873
15874
15875
666
15876
15877
666
15878
666
15879
15880
666
15881
15882
666
15883
666
15884
15885
15886
15887
666
15888
15889
15890
15891
666
15892
15893
15894
15895
15896
666
15897
15898
15899
15900
15901
15902
15903
15904
15905
15906
15907
666
15908
15909
15910
15911
15912
666
15913
15914
666
15915
15916
15917
15918
666
15919
666
15920
15921
15922
15923
15924
15925
15926
15927
15928
15929
15930
15931
15932
15933
15934
15935
666
15936
15937
15938
15939
666
15940
15941
15942
666
15943
15944
15945
15946
15947
15948
15949
15950
15951
15952
15953
15954
15955
666
15956
666
15957
15958
15959
15960
15961
15962
15963
15964
15965
666
15966
15967
15968
15969
15970
15971
15972
15973
15974
15975
15976
666
15977
15978
15979
15980
15981
15982
15983
15984
666
15985
15986
15987
666
15988
15989
15990
15991


IndexError: list index out of range